### This is my original working file where I started working with the data and then went on to try linear regression and the grid search. Originally I had some great results with this, before removing NaN values, probably too good to be true, but when I moved on to the Grid Search I started getting other errors (like NaN values). I'm saving this Juypter Notebook in case I want to go back to an earlier checkpoint for those R2 of 0.99999.....

In [1]:
# Imports
import warnings
warnings.simplefilter('ignore')

# %matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# Read the csv file into a pandas DataFrame
exoplanet_data = pd.read_csv('cumulative.csv')
exoplanet_data.head()

,rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,1,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,2,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,3,10811496,K00753.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,4,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,5,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [3]:
exoplanet_reduced = exoplanet_data.drop(['rowid', 'kepid','kepoi_name', 'koi_pdisposition','kepler_name', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_steff_err2', 'koi_slogg_err1', 'koi_slogg_err2', 'koi_srad_err1', 'koi_srad_err2',], axis=1)
exoplanet_reduced.head()


,koi_disposition,koi_score,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,...,koi_model_snr,koi_tce_plnt_num,koi_tce_delivname,koi_steff,koi_steff_err1,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,CONFIRMED,1.000,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,-0.002160,0.146,...,35.8,1.0,q1_q17_dr25_tce,5455.0,81.0,4.467,0.927,291.93423,48.141651,15.347
1,CONFIRMED,0.969,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,0.586,...,25.8,2.0,q1_q17_dr25_tce,5455.0,81.0,4.467,0.927,291.93423,48.141651,15.347
2,FALSE POSITIVE,0.000,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,0.000581,-0.000581,0.969,...,76.3,1.0,q1_q17_dr25_tce,5853.0,158.0,4.544,0.868,297.00482,48.134129,15.436
3,FALSE POSITIVE,0.000,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,1.276,...,505.6,1.0,q1_q17_dr25_tce,5805.0,157.0,4.564,0.791,285.53461,48.285210,15.597
4,CONFIRMED,1.000,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,-0.001130,0.701,...,40.9,1.0,q1_q17_dr25_tce,6031.0,169.0,4.438,1.046,288.75488,48.226200,15.509


In [4]:
exoplanet_reduced2 = exoplanet_reduced.drop(['koi_fpflag_ec', 'koi_period_err1', 'koi_period_err2','koi_teq_err1', 'koi_teq_err2', 'koi_insol_err1', 'koi_insol_err2', 'koi_tce_plnt_num', 'koi_tce_delivname', 'koi_steff_err1'], axis=1)
exoplanet_reduced2.head()


,koi_disposition,koi_score,koi_period,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,...,koi_prad_err2,koi_teq,koi_insol,koi_model_snr,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,CONFIRMED,1.000,9.488036,170.538750,0.002160,-0.002160,0.146,0.318,-0.146,2.95750,...,-0.15,793.0,93.59,35.8,5455.0,4.467,0.927,291.93423,48.141651,15.347
1,CONFIRMED,0.969,54.418383,162.513840,0.003520,-0.003520,0.586,0.059,-0.443,4.50700,...,-0.19,443.0,9.11,25.8,5455.0,4.467,0.927,291.93423,48.141651,15.347
2,FALSE POSITIVE,0.000,19.899140,175.850252,0.000581,-0.000581,0.969,5.126,-0.077,1.78220,...,-1.31,638.0,39.30,76.3,5853.0,4.544,0.868,297.00482,48.134129,15.436
3,FALSE POSITIVE,0.000,1.736952,170.307565,0.000115,-0.000115,1.276,0.115,-0.092,2.40641,...,-2.83,1395.0,891.96,505.6,5805.0,4.564,0.791,285.53461,48.285210,15.597
4,CONFIRMED,1.000,2.525592,171.595550,0.001130,-0.001130,0.701,0.235,-0.478,1.65450,...,-0.35,1406.0,926.16,40.9,6031.0,4.438,1.046,288.75488,48.226200,15.509


In [5]:
exoplanet_reduced3 = exoplanet_reduced2.drop(['koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact_err1', 'koi_impact_err2', 'koi_duration_err1', 'koi_duration_err2', 'koi_depth_err1', 'koi_depth_err2','koi_prad_err1', 'koi_prad_err2', 'koi_model_snr'], axis=1)
exoplanet_reduced3.head()


,koi_disposition,koi_score,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,CONFIRMED,1.000,9.488036,170.538750,0.146,2.95750,615.8,2.26,793.0,93.59,5455.0,4.467,0.927,291.93423,48.141651,15.347
1,CONFIRMED,0.969,54.418383,162.513840,0.586,4.50700,874.8,2.83,443.0,9.11,5455.0,4.467,0.927,291.93423,48.141651,15.347
2,FALSE POSITIVE,0.000,19.899140,175.850252,0.969,1.78220,10829.0,14.60,638.0,39.30,5853.0,4.544,0.868,297.00482,48.134129,15.436
3,FALSE POSITIVE,0.000,1.736952,170.307565,1.276,2.40641,8079.2,33.46,1395.0,891.96,5805.0,4.564,0.791,285.53461,48.285210,15.597
4,CONFIRMED,1.000,2.525592,171.595550,0.701,1.65450,603.3,2.75,1406.0,926.16,6031.0,4.438,1.046,288.75488,48.226200,15.509


In [6]:
data_binary_encoded = pd.get_dummies(exoplanet_reduced3, columns=['koi_disposition'])
data_binary_encoded.head()

,koi_score,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag,koi_disposition_CANDIDATE,koi_disposition_CONFIRMED,koi_disposition_FALSE POSITIVE
0,1.000,9.488036,170.538750,0.146,2.95750,615.8,2.26,793.0,93.59,5455.0,4.467,0.927,291.93423,48.141651,15.347,0,1,0
1,0.969,54.418383,162.513840,0.586,4.50700,874.8,2.83,443.0,9.11,5455.0,4.467,0.927,291.93423,48.141651,15.347,0,1,0
2,0.000,19.899140,175.850252,0.969,1.78220,10829.0,14.60,638.0,39.30,5853.0,4.544,0.868,297.00482,48.134129,15.436,0,0,1
3,0.000,1.736952,170.307565,1.276,2.40641,8079.2,33.46,1395.0,891.96,5805.0,4.564,0.791,285.53461,48.285210,15.597,0,0,1
4,1.000,2.525592,171.595550,0.701,1.65450,603.3,2.75,1406.0,926.16,6031.0,4.438,1.046,288.75488,48.226200,15.509,0,1,0


In [13]:
X = exoplanet_reduced3[['koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol', 'koi_steff', 'koi_slogg', 'koi_steff', 'koi_srad', 'ra', 'dec', 'koi_kepmag']]
y = exoplanet_reduced3['koi_score'].values.reshape(-1, 1)
print(X.shape, y.shape)


(9564, 15) (9564, 1)


In [37]:
# Exported this dataframe to be used with other models

# Now commented out so it doesn't continue to export it
# exoplanet_reduced3.to_csv('exoplanet_reduced.csv', index=False)

In [29]:
# Drop rows with NaN values
exoplanet_complete = exoplanet_reduced3.dropna()
exoplanet_complete.head()

,koi_disposition,koi_score,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,CONFIRMED,1.000,9.488036,170.538750,0.146,2.95750,615.8,2.26,793.0,93.59,5455.0,4.467,0.927,291.93423,48.141651,15.347
1,CONFIRMED,0.969,54.418383,162.513840,0.586,4.50700,874.8,2.83,443.0,9.11,5455.0,4.467,0.927,291.93423,48.141651,15.347
2,FALSE POSITIVE,0.000,19.899140,175.850252,0.969,1.78220,10829.0,14.60,638.0,39.30,5853.0,4.544,0.868,297.00482,48.134129,15.436
3,FALSE POSITIVE,0.000,1.736952,170.307565,1.276,2.40641,8079.2,33.46,1395.0,891.96,5805.0,4.564,0.791,285.53461,48.285210,15.597
4,CONFIRMED,1.000,2.525592,171.595550,0.701,1.65450,603.3,2.75,1406.0,926.16,6031.0,4.438,1.046,288.75488,48.226200,15.509


In [22]:
X = exoplanet_complete[['koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol', 'koi_steff', 'koi_slogg', 'koi_steff', 'koi_srad', 'ra', 'dec', 'koi_kepmag']]
y = exoplanet_complete['koi_score'].values.reshape(-1, 1)
print(X.shape, y.shape)


(7994, 15) (7994, 1)


In [38]:
# Exported this dataframe to be used with other models

# Now commented out so it doesn't continue to export it
# exoplanet_complete.to_csv('exoplanet_complete.csv', index=False)

In [23]:
from sklearn.model_selection import train_test_split

In [ ]:

X_train_minmax, X_test_minmax, y_train_minmax, y_test_minmax = train_test_split(X, y, random_state=42)

In [25]:
from sklearn.preprocessing import MinMaxScaler
X_minmax = MinMaxScaler().fit(X_train)
y_minmax = MinMaxScaler().fit(y_train)

X_train_minmax = X_minmax.transform(X_train)
X_test_minmax = X_minmax.transform(X_test)
y_train_minmax = y_minmax.transform(y_train)
y_test_minmax = y_minmax.transform(y_test)

In [27]:
from sklearn.datasets import make_regression

n_features = 15
X, y = make_regression(n_samples=7994, n_features=n_features, 
                       n_informative=n_features, random_state=42, 
                       noise=0.5, bias=100.0)
print(X.shape)

(7994, 15)


In [28]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

# Fit our model with train data
model.fit(X_train_minmax, y_train_minmax)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [30]:
# Calculate training score and testing score and print
training_score = model.score(X_train_minmax, y_train_minmax)
testing_score = model.score(X_test_minmax, y_test_minmax)

print(f"R2 training score: {training_score}")
print(f"R2 testing score: {testing_score}")

R2 training score: 0.3071065075123848
R2 testing score: 0.35327120946042684


In [ ]:
# Support vector machine linear classifier
#from sklearn.svm import SVC 
#model = SVC(kernel='linear')

In [32]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
parameters = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
#param_grid = {'C': [1, 5, 10],
#             'gamma': [0.0001, 0.001, 0.01]}
grid = GridSearchCV(model, parameters, verbose=3)

In [33]:
grid.fit(X_train_minmax, y_train_minmax)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.17694752404809955, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.2423895444287404, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.2622246092498137, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=True, normalize=False, score=0.17694752404918146, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=True, normalize=False, score=0.24238954442976512, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=T

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:    0.1s finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'fit_intercept': [True, False], 'normalize': [True, False], 'copy_X': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [34]:
print(grid.best_params_)

{'copy_X': True, 'fit_intercept': True, 'normalize': False}


In [35]:
print(grid.best_score_)

0.22717884564245114
